In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/internal-waves/sample_submission.csv
/kaggle/input/internal-waves/train.csv
/kaggle/input/internal-waves/test.csv
/kaggle/input/internal-waves/solution.csv
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/4353.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/2539.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/4803.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/3217.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/3207.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/2673.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/4747.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/3824.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/4358.png
/kaggle/input/internal-waves/images_train-20240709T094004Z-001/im

In [3]:
!uv pip install -q scikit-learn==1.6.1 imblearn --system

In [6]:
%reload_ext cudf.pandas
import os
import gc
import cv2
import timm
import torch
import datetime
import seaborn as sns
from tqdm import tqdm
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer, roc_auc_score
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, train_test_split

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from joblib import Parallel, delayed
#from imblearn.pipeline import Pipeline
from torch.utils.data import Dataset, DataLoader

/usr/local/lib/python3.11/dist-packages/cudf/pandas/__init__.py:65: UserWarning: cudf.pandas detected an already configured memory resource, ignoring 'CUDF_PANDAS_RMM_MODE'=managed_pool
  warnings.warn(


In [7]:
train = pd.read_csv('/kaggle/input/internal-waves/train.csv')
test = pd.read_csv('/kaggle/input/internal-waves/test.csv')

train['file'] = train['id'].apply(lambda x: f'/kaggle/input/internal-waves/images_train-20240709T094004Z-001/images_train/{x}.png')
test['file'] = test['id'].apply(lambda x: f'/kaggle/input/internal-waves/images_test-20240709T093512Z-001/images_test/{x}.png')

train

,id,ground_truth,file
0,2,1,/kaggle/input/internal-waves/images_train-2024...
1,3,1,/kaggle/input/internal-waves/images_train-2024...
2,4,1,/kaggle/input/internal-waves/images_train-2024...
3,6,1,/kaggle/input/internal-waves/images_train-2024...
4,9,1,/kaggle/input/internal-waves/images_train-2024...
...,...,...,...
3396,4855,0,/kaggle/input/internal-waves/images_train-2024...
3397,4856,0,/kaggle/input/internal-waves/images_train-2024...
3398,4857,0,/kaggle/input/internal-waves/images_train-2024...
3399,4858,0,/kaggle/input/internal-waves/images_train-2024...


In [8]:
test

,id,file
0,1,/kaggle/input/internal-waves/images_test-20240...
1,5,/kaggle/input/internal-waves/images_test-20240...
2,7,/kaggle/input/internal-waves/images_test-20240...
3,8,/kaggle/input/internal-waves/images_test-20240...
4,15,/kaggle/input/internal-waves/images_test-20240...
...,...,...
1454,4844,/kaggle/input/internal-waves/images_test-20240...
1455,4847,/kaggle/input/internal-waves/images_test-20240...
1456,4850,/kaggle/input/internal-waves/images_test-20240...
1457,4852,/kaggle/input/internal-waves/images_test-20240...


In [9]:
class Dataset(Dataset):
    def __init__(self, files, transform):
        self.files = files
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert('RGB')
        img = self.transform(img)
        return img

model = timm.create_model('timm/eva02_large_patch14_448.mim_m38m_ft_in1k', pretrained=True)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)
model = model.to('cuda').eval()
model = torch.nn.DataParallel(model)


dataset = Dataset(train['file'].tolist(), transforms)
data_loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
print( len(dataset), len(data_loader) )
ypred = []
for batch in tqdm(data_loader):
    with torch.no_grad():
        ypred.append( model(batch.to('cuda')).softmax(dim=1).cpu().numpy() )
ypred = np.vstack(ypred)
print(ypred.shape)


dataset = Dataset(test['file'].tolist(), transforms)
data_loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
print( len(dataset), len(data_loader) )
ypred_test = []
for batch in tqdm(data_loader):
    with torch.no_grad():
        ypred_test.append( model(batch.to('cuda')).softmax(dim=1).cpu().numpy() )
ypred_test = np.vstack(ypred_test)
print(ypred_test.shape)



xgb_params = {
    'tree_method':        'hist',
    'random_state':       1,
    'learning_rate':      0.05, 
    'max_depth':          7, 
    'subsample':          0.50,
    'colsample_bytree':   0.75, 
    'device':             'cuda',
}

val_score = cross_val_score(
    estimator=XGBClassifier(**xgb_params, n_estimators=550),
    X=ypred,
    y=train['ground_truth'], 
    cv=StratifiedKFold(5, shuffle=True, random_state=2024),
    scoring = make_scorer(roc_auc_score),
)
np.mean(val_score), val_score

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

3401 54


100%|██████████| 54/54 [08:09<00:00,  9.07s/it]


(3401, 1000)
1459 23


100%|██████████| 23/23 [03:31<00:00,  9.19s/it]


(1459, 1000)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [14:58:44] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


(0.9380407923548514,
 array([0.9500647 , 0.939801  , 0.93703223, 0.92943976, 0.93386628]))

In [10]:
fullmodel = XGBClassifier(**xgb_params, n_estimators=550)
fullmodel.fit(ypred, train['ground_truth'])

test['predicted'] = fullmodel.predict_proba(ypred_test)[:, 1]
test[['id', 'predicted']].to_csv('submission.csv', index=False)